#### We are going to create a widget with Vega in version 2 and version 3 to campare both. 
#### These widgets are from Aqueduct App.

#### Este primer caso es referente al problema que tuvimos en Vega con la representación del eje X en donde el primer valor debía indicarse como "Baseline" y no como "2010" y este eje era "movible" (En un principio se calculaba el mínimo valor de la gráfica en el eje Y y a partir de ese valor, se le colocaba un offset al eje X y allí se pintaba. Esto pintaba de forma erronea en algunos casos).

### Vega 2

`{
  "interaction_config": [
    {
      "config": {
        "table": {
          "headers": ["Category","Variability"],
          "columns": [
            {"key": "impactparameter","parse": {}},
            {
              "parse": {"format": ".2f","suffix": "%"},
              "key": "value"
            }
          ],
          "bisect": "year",
          "title": {"key": "year","parse": {}},
          "from": "tables"
        }
      },
      "type": "signal",
      "name": "tooltip"
    }
  ],
  "signals": [
    {
      "streams": [
        {
          "type": "mousemove",
          "expr": "{ xval: +iscale('x', eventX()), yval: +iscale('y', eventY())}"
        }
      ],
      "name": "onMousemove"
    }
  ],
  "padding": {"bottom": 92,"right": 40,"left": 70,"top": 22},
  "scales": [
    {
      "domain": {"field": "year","data": "tables"},
      "range": "width",
      "zero": false,
      "type": "linear",
      "nice": "year",
      "name": "x"
    },
    {
      "domain": {"field": "value","data": "tables"},
      "range": "height",
      "type": "linear",
      "name": "y"
    },
    {
      "domain": {
        "fields": [{"field": "impactparameter","data": "legend"}]
      },
      "range": "cropCatColor",
      "type": "ordinal",
      "name": "color"
    },
    {
      "padding": 1,
      "points": true,
      "domain": [0,1,2],
      "range": "width",
      "type": "ordinal",
      "name": "legend-series-x"
    },
    {
      "domain": {"field": "impactparameter","data": "legend"},
      "range": [-170,-170,-170,-170],
      "type": "ordinal",
      "name": "legend-series-y"
    },
    {
      "name": "texto",
      "type": "ordinal",
      "range": "width",
      "domain": {"data": "axis-x","field": "y", "sort": {"field": "y", "op": "max"}},
      "points": true
    }
  ],
  "height": 140,
  "marks": [
    {
      "properties": {
        "enter": {
          "fontSize": {"value": 12},
          "align": {"value": "center"},
          "text": {"field": "y"},
          "font": {"value": "Roboto"},
          "fill": {"value": "#758290"},
          "y": {"offset": 20,"value": 150},
          "x": {"scale": "texto","field": "y"}
        }
      },
      "type": "text",
      "from": {"data": "texto"}
    },
    {
      "properties": {
        "enter": {
          "text": {"field": "impactparameter"},
          "fill": {"value": "grey"},
          "y": {"offset": 200,"field": "yPos"},
          "x": {
            "offset": -70,
            "scale": "legend-series-x",
            "field": "xPos"
          }
        }
      },
      "type": "text",
      "from": {"data": "legend"}
    },
    {
      "properties": {
        "enter": {
          "shape": {"value": "square"},
          "text": {"scale": "color","field": "impactparameter"},
          "fill": {"scale": "color","field": "impactparameter"},
          "y": {"offset": 196,"field": "yPos"},
          "x": {
            "offset": -80,
            "scale": "legend-series-x",
            "field": "xPos"
          }
        }
      },
      "type": "symbol",
      "from": {"data": "legend"}
    },
    {
      "marks": [
        {
          "properties": {
            "enter": {
              "strokeWidth": {"value": "2"},
              "stroke": {"scale": "color","field": "impactparameter"},
              "y": {"scale": "y","field": "value"},
              "x": {"scale": "x","field": "year"}
            }
          },
          "type": "line"
        }
      ],
      "type": "group",
      "from": {
        "transform": [{"groupby": ["impactparameter"],"type": "facet"}],
        "data": "tables"
      }
    }
  ],
  "data": [
    {
      "format": {"property": "rows","type": "json"},
      "name": "tables",
      "url": "https://wri-rw.carto.com/api/v2/sql?q=WITH data as ( SELECT impactparameter, value, year FROM combined01_prepared where commodity='Potato' and iso='USA' and impactparameter in ('Net trade') AND scenario = 'SSP2-MIRO' ORDER BY year ASC), baseline AS ( SELECT impactparameter, value as baseline FROM data WHERE year =2005 order by year asc) SELECT data.impactparameter, 100*((value-baseline.baseline)/baseline.baseline) as value, value as raw, year FROM data inner join baseline on data.impactparameter=baseline.impactparameter where year > 2005 and year < 2050 order by year asc, data.impactparameter asc"
    },
    {
      "name": "axis-x",
      "values": [
        {"x": 2010,"y": "Baseline","x2": 2010},
        {"x": 2020,"y": "2020","x2": 2020},
        {"x": 2030,"y": "2030","x2": 2030},
        {"x": 2040,"y": "2040","x2": 2040}
      ]
    },
    {
      "transform": [
        {
          "summarize": [
            {
              "field": "year",
              "ops": ["count"],
              "as": ["counts"]
            }
          ],
          "groupby": ["impactparameter"],
          "type": "aggregate"
        },
        {"type": "rank"},
        {
          "field": "xPos",
          "type": "formula",
          "expr": "(datum.rank - 1) % 3"
        },
        {
          "field": "yPos",
          "type": "formula",
          "expr": "floor(datum.rank / 3.1) * 20"
        }
      ],
      "source": "tables",
      "name": "legend"
    }
  ],
  "axes": [
    {"ticks": {"value": "y"},"scale": "texto","type": "x"},
    {
      "tickSizeEnd": 0,
      "properties": {
        "labels": {
          "baseline": {"value": "bottom"},
          "align": {"value": "center"}
        },
        "grid": {
          "strokeWidth": {"value": 1},
          "stroke": {"value": "#eeeeee"}
        },
        "title": {"x": {"value": -45},"fill": {"value": "#000"}}
      },
      "scale": "y",
      "title": "Change in Net Trade",
      "format": "s",
      "layer": "back",
      "type": "y",
      "grid": true
    }
  ],
  "sqlConfig": [
    {
      "keyParams": [
        {"required": true,"key": "commodity"},
        {"required": true,"key": "iso"}
      ],
      "key": "where"
    }
  ],
  "paramsConfig": [
    {"required": true,"key": "commodity"},
    {"required": true,"key": "iso"},
    {"required": true,"key": "countryName"}
  ]
}`

#### Se solucionó colocando una scale nueva llamada "texto" cuyo dominio era el siguiente: <br>
   ` "domain": {"data": "axis-x","field": "y", "sort": {"field": "y", "op": "max"}} 
   ` <br>
#### Importante el campo "sort" porque la leyenda del eje X está tomando valores numéricos y texto ("Baseline"). 

#### La fuente de datos: `"axis-x"` fue creado (en la parte correspondiente a data) como se define a continuación:
<br>
`
{
      "name": "axis-x",
      "values": [
        {"x": 2010,"y": "Baseline","x2": 2010},
        {"x": 2020,"y": "2020","x2": 2020},
        {"x": 2030,"y": "2030","x2": 2030},
        {"x": 2040,"y": "2040","x2": 2040}
      ]
    }
`
<br>
#### Y luego defino en "Marks": 
`
{"type": "text",
 "from": {"data": "texto"},
 "properties": {
   "enter": {
   "fontSize": {"value": 12},
   "align": {"value": "center"},
   "text": {"field": "y"},
   "font": {"value": "Roboto"},
   "fill": {"value": "#758290"},
   "y": {"offset": 20,"value": 150},
   "x": {"scale": "texto","field": "y"}}
      }
    }
`

<img src="./vega2vsvega3_1.png">

### Vega 3

{
  "$schema": "https://vega.github.io/schema/vega/v3.0.json",
  "width": 500,
  "height": 140,
  "padding": 5,
  "signals": [
    {
      "name": "interpolate",
      "value": "linear",
      "bind": {
        "input": "select",
        "options": [
          "basis",
          "cardinal",
          "catmull-rom",
          "linear",
          "monotone",
          "natural",
          "step",
          "step-after",
          "step-before"
        ]
      }
    }
  ],
  "data": [
    {
      "url": "https://wri-rw.carto.com/api/v2/sql?q=WITH data as ( SELECT impactparameter, value, year FROM combined01_prepared where iso='USA' and commodity='Potato' and impactparameter in ('Net trade') AND scenario = 'SSP2-MIRO' ORDER BY year ASC), baseline AS ( SELECT impactparameter, value as baseline FROM data WHERE year =2005 order by year asc), z as (SELECT data.impactparameter, 100*((value-baseline.baseline)/baseline.baseline) as value, value as raw, year FROM data inner join baseline on data.impactparameter=baseline.impactparameter where year > 2005 and year < 2050 order by year asc, data.impactparameter asc) select impactparameter as c, value as y, year as x from z",
      "name": "table",
      "format": {"type": "json","property": "rows"}
    },
    {
      "values": [
        {"x2": 2010,"y": "Baseline","x": 2005},
        {"x2": 2020,"y": "2020","x": 2020},
        {"x2": 2030,"y": "2030","x": 2030},
        {"x2": 2040,"y": "2040","x": 2040}
      ],
      "name": "axis-x"
    }
  ],
  "scales": [
    {
      "name": "x",
      "type": "point",
      "range": "width",
      "domain": {"data": "table","field": "x"}
    },
    {
      "name": "y",
      "type": "linear",
      "range": "height",
      "nice": true,
      "zero": true,
      "domain": {"data": "table","field": "y"}
    },
    {
      "name": "color",
      "type": "ordinal",
      "range": "category",
      "domain": {"data": "table","field": "c"}
    },
    {
      "name": "texto",
      "type": "point",
      "range": "width",
      "domain": {"data": "axis-x","field": "y"}
    }
  ],
  "axes": [
    {
      "orient": "bottom",
      "scale": "texto",
      "title": "",
      "encode": {
        "ticks": {"update": {"stroke": {"value": "steelblue"}}},
        "labels": {
          "interactive": true,
          "update": {
            "text": {"signal": "datum.value"},
            "fill": {"value": "black"},
            "angle": {"value": 0},
            "fontSize": {"value": 10},
            "align": {"value": "center"},
            "baseline": {"value": "middle"},
            "dx": {"value": 0},
            "dy": {"value": 5}
          }
        },
        "title": {"update": {"fontSize": {"value": 10}}},
        "domain": {
          "update": {
            "stroke": {"value": "#333"},
            "strokeWidth": {"value": 1.5}
          }
        }
      }
    },
    {
      "orient": "left",
      "scale": "y",
      "title": "Change in Net Trade",
      "encode": {
        "ticks": {"update": {"stroke": {"value": "steelblue"}}},
        "labels": {
          "interactive": true,
          "update": {
            "text": {"signal": "datum.value"},
            "fill": {"value": "black"},
            "angle": {"value": 0},
            "fontSize": {"value": 10},
            "align": {"value": "center"},
            "baseline": {"value": "middle"},
            "dx": {"value": -10},
            "dy": {"value": 0}
          }
        },
        "title": {"update": {"fontSize": {"value": 10}}},
        "domain": {
          "update": {
            "stroke": {"value": "#333"},
            "strokeWidth": {"value": 1.5}
          }
        }
      }
    }
  ],
  "marks": [
    {
      "type": "group",
      "from": {
        "facet": {"name": "series","data": "table","groupby": "c"}
      },
      "marks": [
        {
          "type": "line",
          "from": {"data": "series"},
          "encode": {
            "enter": {
              "x": {"scale": "x","field": "x"},
              "y": {"scale": "y","field": "y"},
              "stroke": {"scale": "color","field": "c"},
              "strokeWidth": {"value": 2}
            },
            "update": {
              "interpolate": {"signal": "interpolate"},
              "fillOpacity": {"value": 1}
            },
            "hover": {"fillOpacity": {"value": 0.5}}
          }
        }
      ]
    },
    {
      "interactive": true,
      "encode": {
        "update": {
          "font_size": {"value": 12},
          "angle": {"value": 0},
          "align": {"value": "center"},
          "fill": {"value": "#758290"},
          "y": {"offset": 18,"value": 0,"scale": "y"},
          "x": {"offset": 0,"scale": "x","field": "x2"}
        }
      },
      "type": "text",
      "name": "label",
      "from": {"data": "axis-x"}
    }
  ]
}

<img src="./vega2vsvega3_2.png">

#### En la versión 3 de Vega, defino igualmente mi data y mi scale (el scale no hace falta colocar el sort) y ya NO defino mi eje en marks sino que lo hago directamente en "axes". 